# Improving Training Data for Sentiment Analysis with NLTK

So now it is time to train on a new data set. Our goal is to do **Twitter sentiment**, so we're hoping for a data set that is a bit shorter per positive and negative statement. It just so happens that I have a data set of 5300+ positive and 5300+ negative movie reviews, which are much shorter. These should give us a bit more accuracy from the larger training set, as well as be more fitting for tweets from Twitter.

I have hosted both files here, you can find them by going to the downloads for the short reviews. Save these files as positive.txt and negative.txt.

Now, we can build our new data set in a very similar way as before. What needs to change?



We need a new methodology for creating our "documents" variable, and then we also need a new way to create the "all_words" variable. No problem, really, here's how I did it:

In [3]:
from nltk.tokenize import word_tokenize
import nltk

short_pos = open("positive.txt","r").read()
short_neg = open("negative.txt","r").read()

documents = []

for r in short_pos.split('\n'):
    documents.append( (r, "pos") ) #appends it with a tuple: the text, and the classification

for r in short_neg.split('\n'):
    documents.append( (r, "neg") )


all_words = []

short_pos_words = word_tokenize(short_pos) #tokenizing positive
short_neg_words = word_tokenize(short_neg) #tokenizing negative

for w in short_pos_words: #tokenizing all words from both classes
    all_words.append(w.lower())

for w in short_neg_words:
    all_words.append(w.lower()) #note to self: this is not shuffled

all_words = nltk.FreqDist(all_words) #changes the list of all words to a frequency distribution [note to self: i dont see the classification here]

Next, we also need to adjust our feature finding function, mainly tokenizing by word in the document, since we didn't have a nifty .words() feature for our new sample. I also went ahead and increased the most common words:

In [5]:
word_features = list(all_words.keys())[:5000]
import random

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)

Other than this, the rest is the same. Here's the full script just in case you or I missed something:

This process will take a while.. You may want to just go run some errands. It took me about 30-40 minutes to run it in full, and I am running an i7 3930k. For the typical processor in the year I am writing this (2015), it may be hours. This is a one and done process, however.

In [7]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from nltk.tokenize import word_tokenize


class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
        
short_pos = open("positive.txt","r").read()
short_neg = open("negative.txt","r").read()

documents = []

for r in short_pos.split('\n'):
    documents.append( (r, "pos") )

for r in short_neg.split('\n'):
    documents.append( (r, "neg") )


all_words = []

short_pos_words = word_tokenize(short_pos)
short_neg_words = word_tokenize(short_neg)

for w in short_pos_words:
    all_words.append(w.lower())

for w in short_neg_words:
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:5000]

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

#print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))

featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)

# positive data example:      
training_set = featuresets[:10000]
testing_set =  featuresets[10000:]

##
### negative data example:      
##training_set = featuresets[100:]
##testing_set =  featuresets[:100]


classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

##SVC_classifier = SklearnClassifier(SVC())
##SVC_classifier.train(training_set)
##print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


voted_classifier = VoteClassifier(
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)


Original Naive Bayes Algo accuracy percent: 74.3975903614458
Most Informative Features
              engrossing = True              pos : neg    =     21.0 : 1.0
               inventive = True              pos : neg    =     15.0 : 1.0
              refreshing = True              pos : neg    =     13.7 : 1.0
                powerful = True              pos : neg    =     13.0 : 1.0
                    warm = True              pos : neg    =     11.8 : 1.0
               wonderful = True              pos : neg    =     11.8 : 1.0
               realistic = True              pos : neg    =     11.7 : 1.0
                provides = True              pos : neg    =     11.4 : 1.0
            refreshingly = True              pos : neg    =     11.0 : 1.0
             mesmerizing = True              pos : neg    =     11.0 : 1.0
                delicate = True              pos : neg    =     11.0 : 1.0
                  stupid = True              neg : pos    =     11.0 : 1.0
             

C:\Users\morge\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 65.06024096385542
LinearSVC_classifier accuracy percent: 72.7409638554217
NuSVC_classifier accuracy percent: 72.89156626506023
voted_classifier accuracy percent: 75.30120481927712


In [10]:
#saving the documents
save_docs = open("documents.pickle","wb")
pickle.dump(documents, save_docs)
save_docs.close()

In [11]:
#saving the Naive Bayes:
save_naive = open("naive.pickle","wb")
pickle.dump(classifier, save_naive)
save_naive.close()

In [18]:
# saving mnb
save_MNB = open("mnb.pickle","wb")
pickle.dump(MNB_classifier, save_MNB)
save_MNB.close()

In [19]:
# saving bernoulliNB, logistic, SGDC, LinSVC, NuSVC, and voted:

save_BNB = open("bnb.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_BNB)
save_BNB.close()

save_logreg = open("logreg.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_logreg)
save_logreg.close()

save_SGDC = open("SGDV.pickle","wb")
pickle.dump(SGDClassifier_classifier,save_SGDC)
save_SGDC.close()

save_linSVC = open("linsvc.pickle","wb")
pickle.dump(LinearSVC_classifier, save_linSVC)
save_linSVC.close()

save_NuSVC = open("nusvc.pickle","wb")
pickle.dump(NuSVC_classifier, save_NuSVC)
save_NuSVC.close()

save_voted = open("voted.pickle","wb")
pickle.dump(voted_classifier, save_voted)
save_voted.close()

In [16]:
# and the word_features

save_wordfeatures = open("wordfeautures.pickle","wb")
pickle.dump(word_features, save_wordfeatures)
save_wordfeatures.close()

In [17]:
# and the featuresets

save_featuresets = open("featuresets.pickle","wb")
pickle.dump(featuresets, save_featuresets)
save_featuresets.close()